# Evolver Loop 1 Analysis

Analyzing available solutions and strategies to beat the target of 68.919154

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

def score_submission(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    total_score = Decimal('0')
    per_n_scores = {}
    
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg'])
                     for _, row in group_data.iterrows()]
        side = get_tree_list_side_length(tree_list)
        score = side ** 2 / Decimal(str(n))
        total_score += score
        per_n_scores[n] = float(score)
    
    return float(total_score), per_n_scores

print('Scoring functions defined.')

In [ ]:
# Score the snapshot solution that claimed 70.559
snapshot_path = '/home/nonroot/snapshots/santa-2025/21156850282/submission/submission.csv'
snapshot_score, snapshot_per_n = score_submission(snapshot_path)

print(f'Snapshot solution score: {snapshot_score:.6f}')
print(f'Target score: 68.919154')
print(f'Gap to target: {snapshot_score - 68.919154:.6f}')

In [ ]:
# Score our current baseline
baseline_path = '/home/code/exploration/preoptimized/submission.csv'
baseline_score, baseline_per_n = score_submission(baseline_path)

print(f'Baseline (bucket-of-chump) score: {baseline_score:.6f}')
print(f'Snapshot is better by: {baseline_score - snapshot_score:.6f}')

In [ ]:
# Compare per-N scores to find where improvements are
import matplotlib.pyplot as plt

n_values = sorted(baseline_per_n.keys())
baseline_scores = [baseline_per_n[n] for n in n_values]
snapshot_scores = [snapshot_per_n.get(n, 0) for n in n_values]

# Find N values where snapshot is better
improvements = []
for n in n_values:
    diff = baseline_per_n[n] - snapshot_per_n.get(n, baseline_per_n[n])
    if diff > 0.0001:
        improvements.append((n, diff))

print(f'N values where snapshot is better ({len(improvements)} total):')
for n, diff in sorted(improvements, key=lambda x: -x[1])[:20]:
    print(f'  N={n}: improvement = {diff:.6f}')

In [ ]:
# Analyze the score distribution by N range
ranges = [(1, 50), (51, 100), (101, 150), (151, 200)]

print('Score by N range:')
print('Range\t\tBaseline\tSnapshot\tDiff')
for start, end in ranges:
    baseline_range = sum(baseline_per_n[n] for n in range(start, end+1))
    snapshot_range = sum(snapshot_per_n.get(n, 0) for n in range(start, end+1))
    diff = baseline_range - snapshot_range
    print(f'{start}-{end}\t\t{baseline_range:.4f}\t\t{snapshot_range:.4f}\t\t{diff:.4f}')